In [1]:
!pip install finance-datareader
!pip install pycaret

     |████████████████████████████████| 288 kB 5.4 MB/s 
     |████████████████████████████████| 113 kB 38.9 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 16.9 MB 221 kB/s 
     |████████████████████████████████| 2.0 MB 35.0 MB/s 
     |████████████████████████████████| 1.3 MB 30.6 MB/s 
     |████████████████████████████████| 1.7 MB 34.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 261 kB 37.2 MB/s 
     |████████████████████████████████| 6.8 MB 14.5 MB/s 
     |████████████████████████████████| 167 kB 49.1 MB/s 
     |████████████████████████████████| 62 kB 708 kB/s 
     |████████████████████████████████| 675 kB 38.3 MB/s 
     |████████████████████████████████| 10.1 MB 33.8 MB/s 
     |███████████████████

Reading package lists... Done
Building dependency tree       
Reading state information... Done
scrot is already the newest version (0.8-18).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.6.9-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [92]:
import pandas as pd
import numpy as np
import os
from pycaret.regression import *
import FinanceDataReader as fdr
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from tqdm import tqdm

## Get Stock List

In [93]:
path = ''
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'
stock_data = 'stock_data_until21_11_19(11-23).csv'
merge_stock_data = '20210104-20211119_stock.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_data = pd.read_csv(os.path.join(path,stock_data))
merge_stock_data = pd.read_csv(os.path.join(path,merge_stock_data))
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))


In [94]:
start_date = '20210104'
end_date = '20211119'

In [95]:
# import lightgbm as lgb
# model = LinearRegression()
# for code in tqdm(stock_list['종목코드'].values):
#   data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
#   data = pd.merge(stock_data, data,on='Date',how='left').set_index('Date')
#   X = data[fratures]
#   y = data['Close']
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [96]:
# Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
# code = '005930'
# data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
# data = pd.merge(Business_days, data, how = 'outer')
# data['weekday'] = data.Date.apply(lambda x : x.weekday())
# data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
# data.Close = data.Close.interpolate()


In [97]:
# merge_stock_data.to_datetime()
# data = pd.merge(data, merge_stock_data, on='Date')

In [121]:
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()

sample = pd.merge(stock_data,sample,on='Date',how='left')

train = sample[sample['Date'] < '2021-11-01']
test =  sample[sample['Date'] <= '2021-11-05']
test =  test[test['Date'] >= '2021-11-01']
test = pd.DataFrame(test['Date']).reset_index().drop('index',axis=1)
test

,Date
0,2021-11-01
1,2021-11-02
2,2021-11-03
3,2021-11-04
4,2021-11-05


In [ ]:

features = ['Date','NASDAQ','BTC', 'US2YT','US10YT','Gold','Oil','Close']
train = train[features]
stock_pred_dict = {} 
for code in tqdm(stock_list['종목코드'].values):

  sample_code = stock_list.loc[0,'종목코드']
  sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
  sample = pd.merge(stock_data,sample,on='Date',how='left')

  train = sample[sample['Date'] < '2021-11-01']
  test =  sample[sample['Date'] <= '2021-11-05']
  test =  test[test['Date'] >= '2021-11-01']

  test = pd.DataFrame(test['Date']).reset_index().drop('index',axis=1)
  for i in range(7):
    model=setup(
        data=train,
        target = features[i+1],
        ignore_features = features[i+2:],
        log_experiment = True,
        use_gpu = True
      )
    tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
    tuned_model_ = finalize_model(tuned_model)
    pred = predict_model(tuned_model_, test)
    test[features[i+1]] = pred['Label']
  
  stock_pred_dict[code] = test['Close']


,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.3173,0.1337,0.3656,0.8326,0.0105,0.0094
1,0.3013,0.1303,0.3609,0.8778,0.0101,0.0087
2,0.3761,0.2361,0.4859,0.8561,0.0138,0.0110
3,0.2738,0.1598,0.3998,0.7822,0.0115,0.0081
4,0.1978,0.0512,0.2263,0.7500,0.0065,0.0058
5,0.2914,0.1266,0.3558,0.8504,0.0101,0.0086
6,0.4505,0.4065,0.6375,0.7210,0.0177,0.0130
7,0.3296,0.3293,0.5738,0.7212,0.0156,0.0093
8,0.2570,0.1070,0.3271,0.9015,0.0093,0.0075
9,0.3862,0.2236,0.4729,0.5920,0.0136,0.0114


IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:24:18
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
Date,Date
NASDAQ,Numeric
BTC,Numeric
US2YT,Numeric
US10YT,Numeric
Gold,Numeric
Oil,Label


In [77]:
val_x

,Date,NASDAQ,BTC,US2YT,US10YT,Gold,Oil,Close
0,2021-01-04,13209.357422,4.125685e+07,0.140962,1.067950,35.727725,35.233214,86638.762562
1,2021-01-05,13184.406250,4.081069e+07,0.134639,1.067950,35.506910,35.357242,86638.581947
2,2021-01-06,13164.841797,4.115406e+07,0.134639,1.067950,35.506428,35.357242,86630.257872
3,2021-01-07,13169.666016,4.109090e+07,0.136324,1.067950,35.513694,35.357242,86595.608300
4,2021-01-08,13210.484375,4.140210e+07,0.137751,1.067950,35.568299,35.345436,86597.173577
...,...,...,...,...,...,...,...,...
225,2021-11-15,15892.526367,7.537798e+07,0.474716,1.591043,34.755708,56.320558,70617.298103
226,2021-11-16,15867.575195,7.498689e+07,0.494702,1.591043,34.900623,56.372369,70784.200694
227,2021-11-17,15848.009766,7.524878e+07,0.491403,1.591043,34.816129,56.202088,70552.815681
228,2021-11-18,15852.833984,7.485264e+07,0.480299,1.591043,34.764444,56.282583,70567.928414


In [50]:
tuned_model

Lars(copy_X=True, eps=0.007, fit_intercept=True, fit_path=True, jitter=None,
     n_nonzero_coefs=500, normalize=True, precompute='auto', random_state=2472,
     verbose=False)

In [ ]:
x = sample.iloc[0:-2].to_numpy()
x.shape

(42, 5)

In [ ]:
y = sample.iloc[1:-1].to_numpy()
y_0 = y[:,0]
y_1 = y[:,1]
y_2 = y[:,2]
y_3 = y[:,3]
y_4 = y[:,4]

y_values = [y_0, y_1, y_2, y_3, y_4]

In [ ]:
x_public = sample.iloc[-2].to_numpy()

- 예측

In [ ]:
predictions = []
for y_value in y_values :
    model.fit(x,y_value)
    prediction = model.predict(np.expand_dims(x_public,0))
    predictions.append(prediction[0])
predictions

[70206.67660106532,
 69631.42785252717,
 69062.32129096359,
 69258.21096883612,
 68846.00977524316]

- 실제 Public 값

In [ ]:
sample.iloc[-1].values

array([69900., 71500., 70400., 70600., 70200.])

In [ ]:
sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))

In [ ]:

    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    
    x = data.iloc[0:-2].to_numpy() # 2021년 1월 04일 ~ 2021년 10월 22일까지의 데이터로
    y = data.iloc[1:-1].to_numpy() # 2021년 1월 11일 ~ 2021년 10월 29일까지의 데이터를 학습한다.
    y_0 = y[:,0]
    y_1 = y[:,1]
    y_2 = y[:,2]
    y_3 = y[:,3]
    y_4 = y[:,4]

    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = data.iloc[-2].to_numpy() # 2021년 11월 1일부터 11월 5일까지의 데이터를 예측할 것이다.
    
    predictions = []
    for y_value in y_values :
        model.fit(x,y_value)
        prediction = model.predict(np.expand_dims(x_public,0))
        predictions.append(prediction[0])
    sample_submission.loc[:,code] = predictions * 2
sample_submission.isna().sum().sum()

100%|██████████| 370/370 [00:37<00:00,  9.78it/s]


0

In [ ]:
sample_submission.columns

Index(['Day', '000060', '000080', '000100', '000120', '000150', '000240',
       '000250', '000270', '000660',
       ...
       '330860', '336260', '336370', '347860', '348150', '348210', '352820',
       '357780', '363280', '950130'],
      dtype='object', length=371)

In [ ]:
columns = list(sample_submission.columns[1:])

columns = ['Day'] + [str(x).zfill(6) for x in columns]

sample_submission.columns = columns

In [ ]:
sample_submission.to_csv('BASELINE_Linear.csv',index=False)

In [ ]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27919.530611,34687.673458,60773.779528,142621.815394,104901.698658,16669.447967,47219.595113,85236.833170,103490.352393,...,49749.405974,51984.322942,84384.021784,36846.592704,25721.026664,53328.350326,336697.743579,262257.538308,27176.085090,17382.219194
1,2021-11-02,28750.750484,35032.651375,60312.130021,143012.927861,107216.342323,17001.594758,46672.351191,85360.327648,102788.687368,...,48923.253693,51539.564130,84739.157977,35258.227509,25330.528060,53645.466661,335662.149461,264562.230652,27415.549191,17338.522537
2,2021-11-03,28858.095631,34995.888574,60241.470410,145626.792237,111192.733424,17665.577952,45757.683516,85665.326378,102943.419081,...,48834.428543,49341.153729,85450.806060,35320.479243,25623.131825,55136.121480,329167.718872,264623.119599,27466.446666,17286.577226
3,2021-11-04,28901.301911,34866.098057,59701.569734,145351.536595,109530.741544,17846.022961,46380.152350,85494.864447,99958.476851,...,48954.935640,48626.256108,84823.889868,34343.016848,25878.115140,55964.135511,329482.609718,261821.140588,27594.351745,17247.876124
4,2021-11-05,28152.938540,34873.939080,59968.393160,143505.827198,108761.777883,18078.266972,46975.701291,84943.135732,100294.829339,...,49427.019462,47063.105078,86397.651814,34062.808374,26472.657621,55323.587424,321108.356663,264131.897754,27408.366650,17492.773824
5,2021-11-29,27919.530611,34687.673458,60773.779528,142621.815394,104901.698658,16669.447967,47219.595113,85236.833170,103490.352393,...,49749.405974,51984.322942,84384.021784,36846.592704,25721.026664,53328.350326,336697.743579,262257.538308,27176.085090,17382.219194
6,2021-11-30,28750.750484,35032.651375,60312.130021,143012.927861,107216.342323,17001.594758,46672.351191,85360.327648,102788.687368,...,48923.253693,51539.564130,84739.157977,35258.227509,25330.528060,53645.466661,335662.149461,264562.230652,27415.549191,17338.522537
7,2021-12-01,28858.095631,34995.888574,60241.470410,145626.792237,111192.733424,17665.577952,45757.683516,85665.326378,102943.419081,...,48834.428543,49341.153729,85450.806060,35320.479243,25623.131825,55136.121480,329167.718872,264623.119599,27466.446666,17286.577226
8,2021-12-02,28901.301911,34866.098057,59701.569734,145351.536595,109530.741544,17846.022961,46380.152350,85494.864447,99958.476851,...,48954.935640,48626.256108,84823.889868,34343.016848,25878.115140,55964.135511,329482.609718,261821.140588,27594.351745,17247.876124
9,2021-12-03,28152.938540,34873.939080,59968.393160,143505.827198,108761.777883,18078.266972,46975.701291,84943.135732,100294.829339,...,49427.019462,47063.105078,86397.651814,34062.808374,26472.657621,55323.587424,321108.356663,264131.897754,27408.366650,17492.773824
